In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!git clone https://github.com/UniversalDependencies/UD_Ukrainian-IU

Cloning into 'UD_Ukrainian-IU'...
remote: Enumerating objects: 381, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 381 (delta 23), reused 32 (delta 12), pack-reused 338
Receiving objects: 100% (381/381), 12.36 MiB | 13.37 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [ ]:
!pip install --upgrade spacy

In [ ]:
!python -m spacy convert --converter conllu	/content/UD_Ukrainian-IU/uk_iu-ud-train.conllu /tmp
!python -m spacy convert --converter conllu /content/UD_Ukrainian-IU/uk_iu-ud-test.conllu /tmp
!python -m spacy convert --converter conllu /content/UD_Ukrainian-IU/uk_iu-ud-dev.conllu /tmp

ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (5496 documents): /tmp/uk_iu-ud-train.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (892 documents): /tmp/uk_iu-ud-test.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (672 documents): /tmp/uk_iu-ud-dev.spacy


In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines                                      



In [ ]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 25.14 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.


In [ ]:
!cd fastText && pip install .

Processing /content/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3121756 sha256=21c64f75b081915135f6227cfe8f32133dfedf3cf09ce3104850d18caa01c98c
  Stored in directory: /tmp/pip-ephem-wheel-cache-o1cki07c/wheels/22/04/6e/b3aba25c1a5845898b5871a0df37c2126cb0cc9326ad0c08e7
Successfully built fasttext


In [ ]:
from fasttext import load_model
import argparse
import errno

def convert_bin_to_vec(model, output):
    f = load_model(model)
    words = f.get_words()
    with open(output, "w") as fp_out:
        fp_out.write(str(len(words)) + " " + str(f.get_dimension()) + "\n")
        for w in words:
            v = f.get_word_vector(w)
            vstr = ""
            for vi in v:
                vstr += " " + str(vi)
            try:
                fp_out.write(w + vstr + "\n")
            except IOError as e:
                if e.errno == errno.EPIPE:
                    pass

In [ ]:
!rm -rf /content/SpaCyResults

In [ ]:
import os.path
from glob import glob

!mkdir /content/SpaCyResults/

for f in glob("/gdrive/MyDrive/UberGrid/vectors/*"):
    bf, _ = os.path.splitext(os.path.basename(f))

    if os.path.exists(f"/gdrive/MyDrive/extrinsic/SpacyResults/POS/{bf}"):
        print(f"Skipping {bf} as their already exists")
        continue
    
    os.mkdir(f"/gdrive/MyDrive/extrinsic/SpacyResults/POS/{bf}")


    print(f"Converting vectors from {bf}")

    !rm -rf /content/FastText
    !mkdir /content/FastText
    txt_vectors = os.path.join("/content/FastText/", bf + ".vec")

    convert_bin_to_vec(f, txt_vectors)
    !python -m spacy init vectors uk {txt_vectors} /content/FastText

    !python -m spacy train /gdrive/MyDrive/extrinsic/SpacyResults/POS/pos_tag_config.cfg --output /content/SpaCyResults/{bf} --gpu-id 0
    !rm -rf /content/SpaCyResults/{bf}/model-last
    !mv -v /content/SpaCyResults/{bf}/* /gdrive/MyDrive/extrinsic/SpacyResults/POS/{bf}/

In [ ]:
import re

from csv import DictWriter
from pathlib import Path
import os.path
from glob import glob
import json

mask = r"algo-([^.]+)\.epochs-(\d+)\.subwords-(\d+)\.\.(\d+)\.wordngram-(\d+).neg_sampling-(\d+)"

weighed = r"subword(\d+)-(\d+)\.cbow_weighted.epoch(\d+)"


results_path = Path("/gdrive/MyDrive/SpaCyResults/")

with (results_path / "extrinsic_eval.csv").open("w") as fp_out:
    w = DictWriter(fp_out, fieldnames=["vectors_file", "vectors_params", "ents_f", "ents_p", "ents_r"])

    w.writeheader()

    for f in glob("/gdrive/MyDrive/UberGrid/vectors/*"):
        bf, _ = os.path.splitext(os.path.basename(f))
        meta_file = results_path / bf / "model-best/meta.json"
        if not os.path.exists(meta_file):
            print(f"Cannot find file {meta_file}, skipping")
            continue

        m = re.search(mask, bf)
        if not m or not (re.search(weighed, bf)):
            print(f"Cannot parse filename {bf}")
            continue

        groups = m.groups()

        with open(meta_file, "r") as fp:
            meta = json.load(fp)

        w.writerow({
            "vectors_file": bf,
            "vectors_params": f"{groups[0]};{groups[1]};{groups[2]}-{groups[3]};{groups[4]};{groups[5]}",
            "ents_f": meta["performance"]["ents_f"],
            "ents_p": meta["performance"]["ents_p"],
            "ents_r": meta["performance"]["ents_r"],
        })

Cannot parse filename ubertext.fiction_news_wikipedia.filter_rus+short.tokens.txt.d300.subword2-5.cbow_weighted.epoch5
Cannot parse filename ubertext.fiction_news_wikipedia.filter_rus+short.tokens.txt.d300.subword2-5.cbow_weighted.epoch10
